In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [0]:
spark = SparkSession.builder.getOrCreate()

In [0]:
ruta_curated = "gs://data-lake-afiliados/curated/"
ruta_functional_aniomes = "gs://data-lake-afiliados/functional/aniomes"
ruta_functional_anio= "gs://data-lake-afiliados/functional/anio"

In [0]:
df_curated = spark.read.format("parquet").option("header","true").load(ruta_curated)

In [0]:
df_curated.show()


+-------+---+--------+----------+-------------+----+----------+-----------+------------------+------------+-------------+--------+------------+--------------+----+-----------+---------+--------------------+---------+------+
|periodo| id|  numdoc|first_name|    last_name|edad|tipo_fondo| afp_origen|           empresa|         ruc|tipo_comision|     ram|tipo_usuario|tipo_prospecto|sexo|gest_fecha_| telefono|              correo|   equipo| etapa|
+-------+---+--------+----------+-------------+----+----------+-----------+------------------+------------+-------------+--------+------------+--------------+----+-----------+---------+--------------------+---------+------+
| 202301|  1|86610256|       Ray|  Laird-Craig|  33|   fondo 1|AFP HABITAT|Sunset Enterprises|375810124154|        flujo|52271.47|       nuevo|         grupo|   F|  1/24/2024|924203949|rlairdcraig0@doma...|   LIMA 5|cierre|
| 202301|  2|58943075|    Loydie|        Tyres|  26|   fondo 0|  PROFUTURO| Widget Industries|6073572007

In [0]:
df_curated.createOrReplaceTempView("tb_afiliados")

df_join = spark.sql("SELECT DISTINCT(periodo) FROM tb_afiliados ORDER BY periodo DESC")
df_join.show()

+-------+
|periodo|
+-------+
| 202401|
| 202312|
| 202311|
| 202310|
| 202309|
| 202308|
| 202307|
| 202306|
| 202305|
| 202304|
| 202303|
| 202302|
| 202301|
+-------+



In [0]:
df_curated.write.partitionBy('periodo').parquet(ruta_functional_aniomes, mode='overwrite')

In [0]:
# Extraer el año de la columna 'periodo'
df_curated = df_curated.withColumn('anio', substring(col('periodo'), 1, 4))

# Guardar el DataFrame particionado por los primeros 4 dígitos de 'periodo' en Parquet
df_curated.write.partitionBy('anio').parquet(ruta_functional_anio, mode='overwrite')